# Data Loading

In [1]:
import pandas as pd
import numpy as np

# Read the TSV file into a pandas DataFrame
df = pd.read_csv("/content/drive/MyDrive/Interview Analysis Molecule.tsv", delimiter='\t')

# Set display options to show all columns
pd.set_option('display.max_columns', None)

# Display the first few rows of the df
df.head()


,contract_id,published_date,start_date,duration_extension,outcome,second_place_outcome,buyer,region,atc,duration,contract_type,sku,end_date_extension,participants_no,quantity_annual,quantity_total,maximum_price_allowed,active_ingredient,pack_strength,participants,participants_price,published_date_month,winner,winner_price,second_place,second_place_price
0,1,2013-05-16,2013-05-24,10,won,NaN,buyer_1,region_1,C07AB07,39,regional,molecule_x_10mg_tablet,2017-06-30,1,9860,32966.511085,0.10000,molecule_x,10mg,participants_16,0.0004,2013-05-01,participants_16,0.00040,NaN,NaN
1,2,2013-04-29,2013-06-21,0,won,NaN,buyer_2,region_2,C07AB07,48,wide area,molecule_x_10mg_tablet,2017-06-30,1,54988,220249.442660,63442.21000,molecule_x,10mg,participants_16,0.007,2013-04-01,participants_16,0.00700,NaN,NaN
2,3,2013-06-09,2014-08-14,6,won,lost,buyer_3,region_3,C07AB07,12,regional,molecule_x_10mg_tablet,2016-02-14,2,1904,2680.639219,0.11635,molecule_x,10mg,participants_16|participants_23,0.014|0.07071,2013-06-01,participants_16,0.01400,participants_23,0.07071
3,4,2013-05-04,2013-05-17,19,won,NaN,buyer_4,region_4,C07AB07,36,regional,molecule_x_10mg_tablet,2017-12-31,1,1652,5900.676494,0.03000,molecule_x,10mg,participants_16,0.025,2013-05-01,participants_16,0.02500,NaN,NaN
4,5,2013-12-18,2013-12-17,6,won,NaN,buyer_5,region_5,C07AB07,36,regional,molecule_x_10mg_tablet,2017-06-30,1,1000,3289.223410,0.00010,molecule_x,10mg,participants_16,1e-05,2013-12-01,participants_16,0.00001,NaN,NaN


# Observations

Other than the obvious information such as the dates, regions, etc. the flow of the data in relation to the task appears to be that multiple 'participants' can bid on a contract, and the winner is chosen from here by the lowest bid (as per the task's instructions).

Therefore the primary columns of interest will be:
participants_no, participants, participants_price, winner, and winner_price. It is important to notice that some of these values are stored as lists when participants > 1.

There is also a 'maximum price' column that I may want to check winner prices against, depending on what the values are compared to the winner prices... but without additional context, this could indicate some other type of metric.

I also note an 'outcome' column. Depending on how this column works, it could mean that some contracts that are ongoing may not yet have a winner but, again, without wider context of the data this is unknown.

## Plan / thought process

At first glance, I could say that whenever the number of participants is 1, their bid can be passed straight through to the winning bid.

However, it would be worth checking that in this case, only one participant exists in the 'participants' column, and that only 1 'partcipants_price' exists.

Essentially, whenever information in one column would imply something (such as the number of participants), it is important to check that other columns would "agree" with that information.

# Basic checking of NaN values and types

In [2]:
# Print data columns before proceeding to checks
df.columns

Index(['contract_id', 'published_date', 'start_date', 'duration_extension',
       'outcome', 'second_place_outcome', 'buyer', 'region', 'atc', 'duration',
       'contract_type', 'sku', 'end_date_extension', 'participants_no',
       'quantity_annual', 'quantity_total', 'maximum_price_allowed',
       'active_ingredient', 'pack_strength', 'participants',
       'participants_price', 'published_date_month', 'winner', 'winner_price',
       'second_place', 'second_place_price'],
      dtype='object')

In [3]:
# Check data types
print(df.dtypes)

# Check for missing values
print(df.isnull().sum())

contract_id                int64
published_date            object
start_date                object
duration_extension         int64
outcome                   object
second_place_outcome      object
buyer                     object
region                    object
atc                       object
duration                   int64
contract_type             object
sku                       object
end_date_extension        object
participants_no            int64
quantity_annual            int64
quantity_total           float64
maximum_price_allowed    float64
active_ingredient         object
pack_strength             object
participants              object
participants_price        object
published_date_month      object
winner                    object
winner_price             float64
second_place              object
second_place_price       float64
dtype: object
contract_id               0
published_date            0
start_date                0
duration_extension        0
outcome         

So far, the only concern is that there is one null value for atc. Null values for second place seems to be acceptable based on the structure of the data.

In [4]:
# Filter the DataFrame where 'atc' is null
null_atc_df = df[df['atc'].isnull()]

# Display the filtered DataFrame
null_atc_df


,contract_id,published_date,start_date,duration_extension,outcome,second_place_outcome,buyer,region,atc,duration,contract_type,sku,end_date_extension,participants_no,quantity_annual,quantity_total,maximum_price_allowed,active_ingredient,pack_strength,participants,participants_price,published_date_month,winner,winner_price,second_place,second_place_price
30,31,2023-10-24,2023-09-26,6,won,lost,buyer_8,region_8,NaN,36,regional,molecule_x_10mg_tablet,2027-03-30,4,36360,109912.351249,0.09322,molecule_x,10mg,participants_19|participants_7|participants_16...,0.0155|0.06256|0.08|0.09286,2023-10-01,participants_19,0.0155,participants_7,0.06256


There doesn't appear to be any other concerns with this data, so I will proceed without changing anything. Adding a check to ensure all required columns contain information is the first change I would make to this data pipeline.

# Verifying Participant Data

This section checks that the number of participants matches the number of entries in columns such as participants price and participants and prints the contract IDs of those that do not match.

In [5]:
# Convert 'participants' and 'participants_price' columns to lists
df['participants'] = df['participants'].str.split('|')
# Convert strings to lists and then to floats
df['participants_price'] = df['participants_price'].str.split('|').apply(lambda x: [float(num) for num in x])

# Convert 'participants_no' column to integer
df['participants_no'] = df['participants_no'].astype(int)

# Check if lengths of 'participants' and 'participants_price' lists match 'participants_no'
participants_lengths = df['participants'].apply(len)
participants_price_lengths = df['participants_price'].apply(len)

# Check for mismatches and print warning messages
mismatched_indices = (participants_lengths != df['participants_no']) | (participants_price_lengths != df['participants_no'])
mismatched_contracts = df.loc[mismatched_indices, 'contract_id']
for contract_id in mismatched_contracts:
    participants_no = df.loc[df['contract_id'] == contract_id, 'participants_no'].iloc[0]
    participants_length = len(df.loc[df['contract_id'] == contract_id, 'participants'].iloc[0])
    participants_price_length = len(df.loc[df['contract_id'] == contract_id, 'participants_price'].iloc[0])
    print(f"Warning: Mismatch in number of participants for contract {contract_id}. Expected: {participants_no}, Found: Participants - {participants_length}, Participants Price - {participants_price_length}")


This section is a more formal error check for the above. I have used an assertion rather than an 'if' check as this should never be an issue if everything in the pipeline is running smoothly - the for loop in the above could be seen as redundant; other options would be to output the mismatched contracts if a CSV if the dataframe wasn't empty.

In [6]:
# Assert that the DataFrame has zero rows
assert len(mismatched_contracts) == 0, "DataFrame should have zero rows."

Check that all bids are positive numbers

In [7]:
# Check if all numbers in 'participants_price' column are positive
assert df['participants_price'].apply(lambda x: all(num >= 0 for num in x)).all(), "Not all numbers in 'participants_price' column are positive."


# Calculating Winner & Winner Price and verifying against existing data

In [21]:
import numpy as np

# Determine the maximum length among all lists in 'participants_price' column
max_length = df['participants_price'].apply(len).max()

# Pad or truncate both 'participants' and 'participants_price' lists to ensure uniform length
# This is needed for conversion to numpy array - subarrays must be of the same length
df['participants'] = df['participants'].apply(lambda x: x + [np.nan] * (max_length - len(x)))
df['participants_price'] = df['participants_price'].apply(lambda x: x + [np.nan] * (max_length - len(x)))

# Convert the lists of participant prices to a NumPy array
participant_prices_array = np.array(df['participants_price'].tolist())

# Find the index of the minimum price for each contract, ignoring NaN values
winner_indices = np.nanargmin(participant_prices_array, axis=1)

# Extract the winner and winning price for each contract
# Both of these functions (zip and enumerate) allow for searching through lists of lists
# By searching through "outer" lists and sub-lists
winners = [df['participants'][i][index] for i, index in enumerate(winner_indices)]
# winners = [participants[index] for index, participants in zip(winner_indices, df['participants'])]
winning_prices = [participant_prices_array[i, index] for i, index in enumerate(winner_indices)]

# Add winner and winning price columns to DataFrame
df['winner_calculated'] = winners
df['winner_price_calculated'] = winning_prices

Lastly, the newly calculated columns can be compared to the existing columns to verify the results are correct

In [27]:
# Display the subset of columns related to winners and their calculated values
df_winners = df[['winner', 'winner_calculated', 'winner_price', 'winner_price_calculated']]

# Check if the calculated results match the existing ones
matches_winner = (df_winners['winner'] == df_winners['winner_calculated']).all()
matches_winner_price = (df_winners['winner_price'] == df_winners['winner_price_calculated']).all()

if matches_winner and matches_winner_price:
    print("All calculated results match the existing ones.")
else:
    # Filter DataFrame to show only records where calculated results do not match existing ones
    df_mismatches = df_winners[(df_winners['winner'] != df_winners['winner_calculated']) |
                               (df_winners['winner_price'] != df_winners['winner_price_calculated'])]

    # Display DataFrame showing only records where calculated results do not match existing ones
    print("Calculated results do not match the existing ones:")
    print(df_mismatches.head())

    raise ValueError("Calculated results do not match the existing ones.")



All calculated results match the existing ones.


## Comparing to max price

In [26]:
# Get the maximum price value
max_price = df['maximum_price_allowed']

# Check if all 'winner_price_calculated' values are less than or equal to the maximum price
all_less_or_equal = (df['winner_price_calculated'] <= max_price).all()

if all_less_or_equal:
    print("All 'winner_price_calculated' values are less than or equal to the maximum price.")
else:
    raise ValueError("Some 'winner_price_calculated' values exceed the maximum price.")


All 'winner_price_calculated' values are less than or equal to the maximum price.


# Summary

**Scalability**:
   * The data processing steps have been designed with scalability in mind. For instance, when dealing with large datasets, we have used efficient methods and libraries such as pandas and NumPy to handle data manipulation and calculations.
   * By leveraging vectorized operations and avoiding unnecessary iterations, the code can efficiently handle large volumes of data without sacrificing performance.
   
**Accuracy**:
   * Accuracy has been ensured through rigorous validation and verification steps at each stage of the data pipeline.
   * Verifying data types, checking for missing values, and validating calculated results against existing data.

The checks here can be implemented as part of a continuous checking process, ticking the box for automation.

As discussed, the solution is already scalable as there are no iterative loops.

If they do not already exist, I would implement automatic testing of the input data, and unit testing of core code functionality at the top of the data pipeline.

I would consider using manual checks for data quality, using random samples of data. I would also consider manually checking that calculations provide the expected results. Even in the creation of this script I inspected the sample manually to ensure that the values returned were as expected.

Long term scalability solutions could include cloud based services such as AWS to process larger volumes of data as and when needed without investing heavily into server architecture.